## Imports

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import cv2
import logging
import imutils
from imutils.video import VideoStream
import numpy as np
import time
import os

In [2]:
# Suppresses the tensorflow messages in the prints
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

## Detect Face Mask Function

In [3]:
print('\033[93m[IN PROGRESS] Invoking the Detect and Predict Mask function')
def predict_mask_using_trained_model(frame_of_face, face_network, mask_network):

	# creating a rectangle acc to the dimension of frame_of_face
	(height, width) = frame_of_face.shape[:2]
	rectangle = cv2.dnn.blobFromImage(frame_of_face, 1.0, (224, 224), (104.0, 177.0, 123.0))

	# face_detection is outputted after passing rectangle throuhg the network
	face_network.setInput(rectangle)
	face_detections = face_network.forward()
	print(face_detections.shape)


	faces_list = []
	location_lists = []
	face_mask_pred = []

	for i in range(0, face_detections.shape[2]):
		# getting the probability from detection
		face_detection_confidence = face_detections[0, 0, i, 2]

		# removing weak probabilities by setting a min confidence
		if face_detection_confidence > 0.5:
			bounding_box = face_detections[0, 0, i, 3:7] * np.array([width, height, width, height])
			(startX, startY, endX, endY) = bounding_box.astype("int")

			# bounding_box should be in dimensions of the fraame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(width - 1, endX), min(height - 1, endY))

			# converting face ROI from BGR to RGB
			ROI_face = frame_of_face[startY:endY, startX:endX]
			ROI_face = cv2.cvtColor(ROI_face, cv2.COLOR_BGR2RGB)
			ROI_face = cv2.resize(ROI_face, (224, 224))
			ROI_face = img_to_array(ROI_face)
			ROI_face = preprocess_input(ROI_face)
			
			location_lists.append((startX, startY, endX, endY))
			faces_list.append(ROI_face)

	if len(faces_list) > 0:
		# batch predictions will help in pacing up the inference
		faces_list = np.array(faces_list, dtype="float32")
		face_mask_pred = mask_network.predict(faces_list, batch_size=32)

	# face location and corresponding locations
	return (location_lists, face_mask_pred)

print('\033[92m[COMPLETE] Invoking the Detect and Predict Mask function')

[IN PROGRESS] Invoking the Detect and Predict Mask function
[COMPLETE] Invoking the Detect and Predict Mask function


## Loading Face Detector Model from Disk

In [4]:
print('\033[93m[IN PROGRESS] Loading model from file')

# load our serialized face detector model from disk
prototxtPath = r"face_detector/deploy.prototxt"
weightsPath = r"face_detector/res10_300x300_ssd_iter_140000.caffemodel"
face_network = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
mask_network = load_model("mask_detector.model")
print('\033[92m[COMPLETE] Loading model from file')

[IN PROGRESS] Loading model from file
[COMPLETE] Loading model from file


## Initialising Video Stream

In [5]:
print('\033[93m[IN PROGRESS] Starting Video Stream')
# Using the default 0 video camera
vs = VideoStream(src=0).start()

# looping every frame from video
while True:
	# resize frame to 400px width
	frame_of_face = vs.read()
	frame_of_face = imutils.resize(frame_of_face, width=400)

	# detecting face and determining if mask is on
	(location_lists, face_mask_pred) = predict_mask_using_trained_model(frame_of_face, face_network, mask_network)

	for (bounding_box, pred) in zip(location_lists, face_mask_pred):

		(noMask, cloth, N95, N95_valve, surgical) = pred
		(startX, startY, endX, endY) = bounding_box
		
		# bounding_box text and color
		if max(N95, noMask, cloth, N95_valve, surgical) == N95:
			label = "Cloth" # Lime
			color = (0, 255, 0)
		elif max(N95, noMask, cloth, N95_valve, surgical) == surgical:
			label = "Surgical"
			color = (255, 0, 255) # Pink
		elif max(N95, noMask, cloth, N95_valve, surgical) == cloth:
			label = "N95"
			color = (255, 0, 0) # Blue
		elif max(N95, noMask, cloth, N95_valve, surgical) == N95_valve:
			label = "No Mask"
			color = (0, 0, 255) # Red
		else:
			label = "N95 Valve"
			color = (255, 255, 0) # Yellow

		# probability on box label
		label = "{}: {:.2f}%".format(label, max(N95, noMask, cloth, N95_valve, surgical) * 100)

		cv2.putText(frame_of_face, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.47, color, 2)
		cv2.rectangle(frame_of_face, (startX, startY), (endX, endY), color, 2)

	# output result
	cv2.imshow("Frame", frame_of_face)

	# Only want 8 bits to check ASCII keypress
	if cv2.waitKey(1) & 0xFF == ord("q"):
		break

# cleaning and stopping videoStream
cv2.destroyAllWindows()
vs.stop()
print('\033[92m[COMPLETE] Program Ends')

[IN PROGRESS] Starting Video Stream
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 488ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 23ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 24ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 21ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 22ms/

: 